### Pip installs

In [ ]:
!pip install langchain_core
!pip install langchain_groq

In [ ]:
pip install configparser

### Imports

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
import psycopg2
import configparser

### DB Connection and getting db schema

In [ ]:
# Function to establish connection with PostgreSQL database using psycopg2
def connect_to_db(host: str, port: str, user: str, passwd: str, db_name: str):
    conn = psycopg2.connect(
        host=host,
        port=port,
        user=user,
        password=passwd,
        database=db_name
    )
    return conn

In [ ]:
# Function to fetch table and column information from the database schema
def get_db_schema(conn, table_name):
    cursor = conn.cursor()
    cursor.execute(f"""
        SELECT column_name
        FROM information_schema.columns
        WHERE table_schema='public' AND table_name='{table_name}'
    """)
    schema_info = [row[0] for row in cursor.fetchall()]
    cursor.close()
    return schema_info

### Creating and Executing SQL Query

In [ ]:
# Function to create SQL query generation chain
def create_sql_chain(conn, target_table, question, groq_api_key):
    schema_info = get_db_schema(conn, target_table)

    template = f"""
        Based on the table schema of table '{target_table}', write a SQL query to answer the question.
        Only provide the SQL query, without any additional text or characters.

        Table schema: {schema_info}
        Question: {question}

        SQL Query:
    """
    prompt = ChatPromptTemplate.from_template(template=template)
    llm = ChatGroq(model="llama3-8b-8192", temperature=0.2, groq_api_key=groq_api_key)

    return (
        RunnablePassthrough(assignments={"schema": schema_info, "question": question})
        | prompt
        | llm
        | StrOutputParser()
    )

In [ ]:
# Function to execute SQL query on the database and fetch results
def execute_sql_query(conn, sql_query):
    cursor = conn.cursor()
    cursor.execute(sql_query)
    results = cursor.fetchall()
    cursor.close()
    return results

### Getting the answer in natural language

In [ ]:
# Function to create natural language response based on SQL query results
def create_nlp_answer(conn, sql_query, results, groq_api_key):
    results_str = "\n".join([str(row) for row in results])

    template = f"""
        Based on the results of the SQL query '{sql_query}', write a natural language response.

        Query Results:
        {results_str}
    """
    prompt = ChatPromptTemplate.from_template(template=template)
    llm = ChatGroq(model="llama3-8b-8192", temperature=0.2, groq_api_key=groq_api_key)

    return (
        RunnablePassthrough(assignments={"sql_query": sql_query, "results": results_str})
        | prompt
        | llm
        | StrOutputParser()
    )

### main function

In [ ]:
def main():
    hostname = config['database']['hostname']
    username = config['database']['username']
    password = config['database']['password']
    database_name = config['database']['database_name']
    port = config['database']['port']
    target_table = config['database']['target_table']
    groq_api_key = config['api']['groq_api_key']
    
    conn = connect_to_db(hostname, port, username, password, database_name)
    print("Connected to the database successfully!")

    while True:
        user_query = input(f"Ask your database a question about {target_table} (or type 'exit', 'quit', or '1' to stop): ")
        if user_query.lower() in ['exit', 'quit', '1']:
            print("Exiting the loop. Goodbye!")
            break

        sql_chain = create_sql_chain(conn, target_table, user_query, groq_api_key)
        sql_query_response = sql_chain.invoke({})
        sql_query = sql_query_response.strip()
        print(f"Generated SQL Query:\n{sql_query}")

        results = execute_sql_query(conn, sql_query)
        if results:
            for row in results:
                print(row)
            # Generate natural language response
            nlp_chain = create_nlp_answer(conn, sql_query, results, groq_api_key)
            nlp_response = nlp_chain.invoke({})
            print(f"Natural Language Response:\n{nlp_response}")
        else:
            print("No results found or error occurred.")

    conn.close()

if __name__ == "__main__":
    main()